## Processing Sequences Using RNNs and CNNs

### Forecasting a Time Series

In [13]:
import tensorflow as tf
import pandas as pd
from pathlib import Path

path = Path('./CTA_-_Ridership_-_Daily_Boarding_Totals_20240612.csv')
df = pd.read_csv(path, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"]
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)
df = df.drop_duplicates()

df

,day_type,bus,rail
date,,,
2001-01-01,U,297192,126455
2001-01-02,W,780827,501952
2001-01-03,W,824923,536432
2001-01-04,W,870021,550011
2001-01-05,W,890426,557917
...,...,...,...
2024-03-27,W,515194,364672
2024-03-28,W,513043,374590
2024-03-29,W,480044,313507


### The ARMA Model Family

In [7]:
%pip install --upgrade pip
%pip install statsmodels

import tensorflow as tf
import pandas as pd
from pathlib import Path
from statsmodels.tsa.arima.model import ARIMA

path = Path('./CTA_-_Ridership_-_Daily_Boarding_Totals_20240612.csv')
df = pd.read_csv(path, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"]
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)
df = df.drop_duplicates()

# origin, today = "2019-01-01", "2019-05-31"
# rail_series = df.loc[origin:today]["rail"].asfreq("D")
# model = ARIMA(
#     rail_series, 
#     order=(1, 0, 0),
#     seasonal_order=(0, 1, 1, 7)
# )

# model = model.fit()
# y_pred = model.forecast()

origin, start_date, end_date = "2019-01-01", "2019-03-01", "2019-05-31"
time_period = pd.date_range(start=start_date, end=end_date)
rail_series = df.loc[origin:end_date]["rail"].asfreq("D")

y_preds = []

for today in time_period.shift(-1):
    model = ARIMA(
        rail_series[origin:today], 
        order=(1, 0, 0),
        seasonal_order=(0, 1, 1, 7)
    )
    model = model.fit()
    y_pred = model.forecast()[0]
    y_preds.append(y_pred)

y_preds = pd.Series(y_preds, index=time_period)
mae = (y_preds - df.loc[origin:end_date]["rail"]).abs().mean()

mae

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
/tmp/ipykernel_40/3994456884.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_pred = model.forecast()[0]
/tmp/ipykernel_40/3994456884.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_pred = model.forecast()[0]
/tmp/ipykernel_40/3994456884.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  

32040.720089453385

### Preparing the Data for Machine Learning Models

In [19]:
import tensorflow as tf
import pandas as pd
from pathlib import Path

# def to_towindows(dataset, size):
#     dataset = dataset.window(size, shift=1, drop_remainder=True)
#     dataset = dataset.flat_map(lambda window: window.batch(size))
#     return dataset

# dataset = to_towindows(tf.data.Dataset.range(6), 4)
# dataset = dataset.map(lambda window: (window[:-1], window[-1]))
# for batch in dataset:
#     print(batch)

path = Path('./CTA_-_Ridership_-_Daily_Boarding_Totals_20240612.csv')
df = pd.read_csv(path, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"]
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)
df = df.drop_duplicates()

rail_train = df["rail"]["2016-01":"2018-12"] / 1e6
rail_valid = df["rail"]["2019-01":"2019-05"] / 1e6
rail_test = df["rail"]["2019-06":] / 1e6

seq_length = 56
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_train.to_numpy(),
    targets=rail_train[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_valid.to_numpy(),
    targets=rail_valid[seq_length:],
    sequence_length=seq_length,
    batch_size=32
)


(<_BatchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
 <_BatchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>)

### Forecasting Using a Linear Model

In [21]:
import tensorflow as tf
import pandas as pd
from pathlib import Path

path = Path('./CTA_-_Ridership_-_Daily_Boarding_Totals_20240612.csv')
df = pd.read_csv(path, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"]
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)
df = df.drop_duplicates()

rail_train = df["rail"]["2016-01":"2018-12"] / 1e6
rail_valid = df["rail"]["2019-01":"2019-05"] / 1e6
rail_test = df["rail"]["2019-06":] / 1e6

seq_length = 56
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_train.to_numpy(),
    targets=rail_train[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_valid.to_numpy(),
    targets=rail_valid[seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=[seq_length]),
])

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    patience=50, 
    restore_best_weights=True
)
opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
history = model.fit(train_ds, epochs=500, validation_data=valid_ds, callbacks=[early_stopping_cb])

Epoch 1/500
33/33 [==============================] - 1s 7ms/step - loss: 0.3105 - mae: 0.6517 - val_loss: 0.0287 - val_mae: 0.2109
Epoch 2/500
33/33 [==============================] - 0s 4ms/step - loss: 0.0161 - mae: 0.1350 - val_loss: 0.0071 - val_mae: 0.0883
Epoch 3/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0089 - mae: 0.0929 - val_loss: 0.0063 - val_mae: 0.0767
Epoch 4/500
33/33 [==============================] - 0s 4ms/step - loss: 0.0075 - mae: 0.0835 - val_loss: 0.0070 - val_mae: 0.0896
Epoch 5/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0075 - mae: 0.0885 - val_loss: 0.0052 - val_mae: 0.0697
Epoch 6/500
33/33 [==============================] - 0s 4ms/step - loss: 0.0061 - mae: 0.0761 - val_loss: 0.0050 - val_mae: 0.0687
Epoch 7/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0058 - mae: 0.0747 - val_loss: 0.0053 - val_mae: 0.0742
Epoch 8/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0055 - m

### Forecasting Using a Simple RNN

In [11]:
import tensorflow as tf
import pandas as pd
from pathlib import Path

path = Path('./CTA_-_Ridership_-_Daily_Boarding_Totals_20240612.csv')
df = pd.read_csv(path, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"]
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)
df = df.drop_duplicates()

rail_train = df["rail"]["2016-01":"2018-12"] / 1e6
rail_valid = df["rail"]["2019-01":"2019-05"] / 1e6
rail_test = df["rail"]["2019-06":] / 1e6

seq_length = 56
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_train.to_numpy(),
    targets=rail_train[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_valid.to_numpy(),
    targets=rail_valid[seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 1]),
])
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    patience=50, 
    restore_best_weights=True
)
opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
history = model.fit(train_ds, epochs=500, validation_data=valid_ds, callbacks=[early_stopping_cb])



Epoch 1/500
33/33 [==============================] - 1s 17ms/step - loss: 0.2281 - mae: 0.5998 - val_loss: 0.1718 - val_mae: 0.5170
Epoch 2/500
33/33 [==============================] - 0s 10ms/step - loss: 0.1454 - mae: 0.4450 - val_loss: 0.0979 - val_mae: 0.3478
Epoch 3/500
33/33 [==============================] - 0s 12ms/step - loss: 0.0795 - mae: 0.3132 - val_loss: 0.0543 - val_mae: 0.2574
Epoch 4/500
33/33 [==============================] - 0s 11ms/step - loss: 0.0458 - mae: 0.2402 - val_loss: 0.0342 - val_mae: 0.2079
Epoch 5/500
33/33 [==============================] - 0s 10ms/step - loss: 0.0304 - mae: 0.1988 - val_loss: 0.0255 - val_mae: 0.1821
Epoch 6/500
33/33 [==============================] - 0s 11ms/step - loss: 0.0235 - mae: 0.1777 - val_loss: 0.0205 - val_mae: 0.1631
Epoch 7/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0203 - mae: 0.1667 - val_loss: 0.0194 - val_mae: 0.1549
Epoch 8/500
33/33 [==============================] - 0s 12ms/step - loss: 0.

### Forecasting Using a Deep RNN

In [15]:
import tensorflow as tf
import pandas as pd
from pathlib import Path

path = Path('./CTA_-_Ridership_-_Daily_Boarding_Totals_20240612.csv')
df = pd.read_csv(path, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"]
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)
df = df.drop_duplicates()

rail_train = df["rail"]["2016-01":"2018-12"] / 1e6
rail_valid = df["rail"]["2019-01":"2019-05"] / 1e6
rail_test = df["rail"]["2019-06":] / 1e6

seq_length = 56
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_train.to_numpy(),
    targets=rail_train[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_valid.to_numpy(),
    targets=rail_valid[seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[None, 1]),
    tf.keras.layers.SimpleRNN(32, return_sequences=True),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(1),
])

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    patience=50, 
    restore_best_weights=True
)
opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])

history = model.fit(train_ds, epochs=500, validation_data=valid_ds, callbacks=[early_stopping_cb])

Epoch 1/500
33/33 [==============================] - 3s 42ms/step - loss: 0.0517 - mae: 0.1897 - val_loss: 0.0041 - val_mae: 0.0672
Epoch 2/500
33/33 [==============================] - 1s 25ms/step - loss: 0.0053 - mae: 0.0705 - val_loss: 0.0022 - val_mae: 0.0442
Epoch 3/500
33/33 [==============================] - 1s 32ms/step - loss: 0.0052 - mae: 0.0717 - val_loss: 0.0045 - val_mae: 0.0759
Epoch 4/500
33/33 [==============================] - 1s 27ms/step - loss: 0.0054 - mae: 0.0726 - val_loss: 0.0082 - val_mae: 0.1142
Epoch 5/500
33/33 [==============================] - 1s 27ms/step - loss: 0.0048 - mae: 0.0682 - val_loss: 0.0031 - val_mae: 0.0549
Epoch 6/500
33/33 [==============================] - 1s 26ms/step - loss: 0.0036 - mae: 0.0526 - val_loss: 0.0021 - val_mae: 0.0404
Epoch 7/500
33/33 [==============================] - 1s 29ms/step - loss: 0.0038 - mae: 0.0572 - val_loss: 0.0025 - val_mae: 0.0462
Epoch 8/500
33/33 [==============================] - 1s 34ms/step - loss: 0.

### Implement a custom RNN cell

In [ ]:
import tensorflow as tf

class LNSimpleRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = tf.keras.layers.SimpleRNNCell(units, activation=None)
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.activation = tf.keras.activations.get(activation)
        
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=968d3c27-50e7-4d42-bdd9-442f6904c1c2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>